In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from fastai.vision.all import *
from fastai.distributed import *
from fastai.metrics import error_rate
from fastai.callback.tracker import SaveModelCallback

from fastai.callback.cutmix import CutMix
from fastai.callback.mixup import MixUp

if torch.cuda.is_available():
  print("Great! Good to go!")
else:
  print('CUDA is not up!')

Great! Good to go!


In [3]:
from gepcore.utils import cell_graph, convolution
from gepcore.entity import Gene, Chromosome
from gepcore.symbol import PrimitiveSet
from gepnet.model_v2 import get_net, arch_config
from gepnet.utils import count_parameters
torch.backends.cudnn.benchmark = True

In [4]:
from pygraphviz import AGraph
import glob

In [19]:
# get chromosme from fil1
gpath = '/home/cliff/ResearchProjects/geppy_nn/mlj_experiments/431'
graphs = [AGraph(g) for g in glob.glob(gpath+'/*.dot')]
_, comp_graphs = cell_graph.generate_comp_graph(graphs)

#cell_graph.draw_graph(graph, 'nb_graphs/rs/run_4')
# print(comp_graphs)

In [ ]:
# generate new chromosome
# define primitive set
pset = PrimitiveSet('cnn')

# add cellular encoding program symbols
pset.add_program_symbol(cell_graph.end)
pset.add_program_symbol(cell_graph.seq)
pset.add_program_symbol(cell_graph.cpo)
pset.add_program_symbol(cell_graph.cpi)

# add convolutional operations symbols
conv_symbol = convolution.get_symbol()
# pset.add_cell_symbol(conv_symbol.conv3x3)
pset.add_cell_symbol(conv_symbol.sepconv3x3)
pset.add_cell_symbol(conv_symbol.dilconv3x3)
pset.add_cell_symbol(conv_symbol.dilconv5x5)
pset.add_cell_symbol(conv_symbol.sepconv5x5)
#pset.add_cell_symbol(conv_symbol.dwconv3x3)
pset.add_cell_symbol(conv_symbol.maxpool3x3)
# pset.add_cell_symbol(conv_symbol.conv1x1)
#pset.add_cell_symbol(conv_symbol.conv3x1x3)
# pset.add_cell_symbol(conv_symbol.conv1x3)
# pset.add_cell_symbol(conv_symbol.conv3x1)

# comp_graphs = []
# for i in range(3):
#     def gene_gen():
#         return Gene(pset, 2)
#     ch = Chromosome(gene_gen, 3)
#     graph, comp_graph = cell_graph.generate_comp_graph(ch)
#     comp_graphs.append(comp_graph)
#     cell_graph.save_graph(graph, 'comp_graphs/tevc_2020/{}'.format(i))
#     cell_graph.draw_graph(graph, 'comp_graphs/tevc_2020/{}'.format(i))

# for i in range(10):
def gene_gen():
    return Gene(pset, 2)
ch = Chromosome(gene_gen, 3)
graphs, comp_graphs = cell_graph.generate_comp_graph(ch)
cell_graph.save_graph(graphs, 'comp_graphs/ssci_2020/')
cell_graph.draw_graph(graphs, 'comp_graphs/ssci_2020/')

In [6]:
seed = 331
np.random.seed(seed)
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [20]:
conf = arch_config(comp_graphs=comp_graphs,
                   channels=16,
                   repeat_list=[3, 4, 4, 1],
                   classes=45)

net = get_net(conf)
count_parameters(net)

5.499773

In [21]:
from ptflops import get_model_complexity_info
# flops, params = get_model_complexity_info(net, (3, 32, 32), as_strings=True, print_per_layer_stat=False)
# print('flops:  ', flops)
# print('params: ', params)
# print()
flops, params = get_model_complexity_info(net, (3, 224, 224), as_strings=False, print_per_layer_stat=False,
                                         verbose=False)
print('flops:  ', flops/1e10)
print('params: ', params/1e6)

flops:   0.7278692608
params:  5.499773


In [ ]:
# x = torch.randn((1, 3, 224, 224))
# def get_layers(model):
#     for name, layer in model._modules.items():
#         if isinstance(layer, nn.Sequential):
#             get_layers(layer)
#         else:
#             layer.register_forward_hook(hooker)

# def hooker(model, input, output):
#     print(model)
#     print('in_shape:', input[0].size())
#     print('out_shape', output.data.size())
#     print('-' * 40)

# get_layers(model)
# model.eval()
# y = model(x)

In [ ]:
#planet_source = untar_data(URLs.PLANET_SAMPLE)
#df = pd.read_csv(planet_source/'labels.csv')
planet_source = Path.home()/'rs_imagery/train-jpg'
df = pd.read_csv(planet_source/'train_label/train_label.csv')

batch_tfms = aug_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0., size=224)

planet = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                   get_x=ColReader(0, pref=f'{planet_source}/', suff='.jpg'),
                   splitter=RandomSplitter(),
                   get_y=ColReader(1, label_delim=' '),
                   batch_tfms = batch_tfms)

data = planet.dataloaders(df, bs=8)

path = Path("/home/cliff/rs_imagery/AID")
tfms = get_transforms(flip_vert=True, max_lighting=0.1, max_zoom=1.05, max_warp=0.)

bs = 128
data = (ImageList.from_folder(path/'train')
        .split_by_rand_pct(valid_pct=0.1, seed=seed) 
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize())

In [ ]:
test_source = Path.home()/'rs_imagery/test-jpg'
df = pd.read_csv(test_source/'test_label/test_label.csv')

batch_tfms = aug_transforms(do_flip=False, max_lighting=0.1, max_zoom=1.05, max_warp=0., size=224)


test = DataBlock(blocks=(ImageBlock, MultiCategoryBlock),
                        get_x=ColReader(0, pref=f'{test_source}/', suff='.jpg'),
                        get_y=ColReader(1, label_delim=' '),
                        batch_tfms = batch_tfms)

ts = test.dataloaders(df, bs=8)

In [ ]:
from fastai.metrics import accuracy_multi

In [ ]:
#model_dir = '/home/cliff/ResearchProjects/models/random_search/'
# learn = Learner(data, net, metrics=[error_rate, accuracy], model_dir=gpath).mixup()

save = SaveModelCallback(monitor='accuracy_multi', name='multi_model')
mixup = MixUp()

learn = Learner(data, net, metrics=partial(accuracy_multi, thresh=0.5), cbs=[save, mixup], 
                loss_func = BCEWithLogitsLossFlat(), model_dir=gpath)
learn.to_fp16()

In [ ]:
learn.lr_find()

# learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(100, 1e-3, wd=1e-4) 

In [ ]:
learn.get_preds(dl=t)

In [ ]:
learn.validate(dl=data.valid)

In [ ]:
################################# Testing ########################################

In [ ]:
gpath = '/home/cliff/ResearchProjects/geppy_nn/mlj_experiments/3-2-train/3-2-seed-2/best/indv_10'
graph = [AGraph(g) for g in glob.glob(gpath+'/*.dot')]
_, comp_graph = cell_graph.generate_comp_graph(graph)


conf = arch_config(comp_graph=comp_graph,
               depth_coeff=1.0,
               width_coeff=1.0,
               channels=16,
               repeat_list=[3, 2, 1, 2],
               classes=30)

net = get_gepnet(conf)

print(count_parameters(net),'\n')


tfms = get_transforms(do_flip=False)
path = Path("/home/cliff/rs_imagery/AID/")

bs = 128

data = (ImageList.from_folder(path)
        .split_by_folder(train='train', valid='test')
        .label_from_folder()
        .transform(tfms, size=224)
        .databunch(bs=bs, num_workers=num_cpus())
        .normalize())

model = Learner(data, net, metrics=[accuracy, error_rate]).load(gpath+'/aid-model-s322')
_, acc, _ = model.validate()
print('%.2f' %(acc.item()*100))